# analysis of altimetric data

In [7]:
import pandas as pd
import xarray as xr

#from h3 import h3
import pyinterp

import gcsfs.mapping
fs = gcsfs.GCSFileSystem(project='pangeo-cnes', token=None)
gcsmap = gcsfs.mapping.GCSMap('pangeo-cnes/alti/tp', gcs=fs)
ds = xarray.open_zarr(gcsmap)

In [4]:
ds

<xarray.Dataset>
Dimensions:         (time: 157384325)
Coordinates:
    latitude        (time) float64 dask.array<chunksize=(13107200,), meta=np.ndarray>
    longitude       (time) float64 dask.array<chunksize=(13107200,), meta=np.ndarray>
  * time            (time) datetime64[ns] 1992-12-31T23:20:35.775270144 ... 2002-04-24T07:56:09.359487744
Data variables:
    cycle           (time) int16 dask.array<chunksize=(13107200,), meta=np.ndarray>
    dac             (time) float32 dask.array<chunksize=(13107200,), meta=np.ndarray>
    lwe             (time) float32 dask.array<chunksize=(13107200,), meta=np.ndarray>
    mdt             (time) float32 dask.array<chunksize=(13107200,), meta=np.ndarray>
    ocean_tide      (time) float64 dask.array<chunksize=(13107200,), meta=np.ndarray>
    sla_filtered    (time) float32 dask.array<chunksize=(13107200,), meta=np.ndarray>
    sla_unfiltered  (time) float32 dask.array<chunksize=(13107200,), meta=np.ndarray>
    track           (time) int16 dask.

In [5]:
ds = ds.isel(time=slice(0,int(1e6)))
print('Dataset size = %.1f GB' %(ds.nbytes/1e9))

df = ds.to_dataframe()
df

Dataset size = 0.1 GB


,cycle,dac,latitude,longitude,lwe,mdt,ocean_tide,sla_filtered,sla_unfiltered,track
time,,,,,,,,,,
1992-12-31 23:20:35.775270144,11,0.2033,65.681052,327.599941,-0.064,-0.294,0.5490,-0.089,-0.152,18
1992-12-31 23:20:36.853850112,11,0.2044,65.670173,327.731917,-0.064,-0.297,0.5468,-0.087,-0.013,18
1992-12-31 23:20:37.932430080,11,0.2062,65.659172,327.863779,-0.064,-0.299,0.5447,-0.084,-0.059,18
1992-12-31 23:20:39.011010048,11,0.2081,65.648050,327.995525,-0.064,-0.300,0.5426,-0.081,-0.119,18
1992-12-31 23:20:40.089590272,11,0.2090,65.636807,328.127153,-0.064,-0.302,0.5401,-0.078,-0.100,18
...,...,...,...,...,...,...,...,...,...,...
1993-01-21 15:46:01.314065920,13,0.1338,58.036213,321.114859,-0.015,-0.574,-0.1564,-0.097,-0.079,39
1993-01-21 15:46:02.392645888,13,0.1343,58.073186,321.193158,-0.015,-0.575,-0.1531,-0.103,-0.089,39
1993-01-21 15:46:03.471226112,13,0.1348,58.110106,321.271627,-0.015,-0.575,-0.1498,-0.109,-0.065,39


---
## pyinterp approach

[pyinterp doc](https://pangeo-pyinterp.readthedocs.io/en/latest/examples.html)

In [8]:
binning = pyinterp.Binning2D(
    pyinterp.Axis(np.arange(-180, 180, 5), is_circle=True),
    pyinterp.Axis(np.arange(-90, 95, 5)))

AttributeError: module 'pyinterp' has no attribute 'Binning2D'

In [ ]:
mx, my = np.meshgrid(ds.lon, ds.lat)
binning.push(mx, my, ds.mss, simple=True)

In [ ]:
binning.variable('mean')

---
## h3

In [ ]:
resolution = 7
%time df["hex_id"] = df.apply(lambda row: h3.geo_to_h3(row["latitude"], row["longitude"], resolution), axis = 1)

In [ ]:
df

In [ ]:
df_stats = df.groupby('hex_id').mean()